<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

In [ ]:
import numpy as np
import pandas as pd
%pylab inline
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold

**Данные Samsung Human Activity Recognition. [Скачать](https://yadi.sk/d/ha-lprkDvozGx)**

In [ ]:
# поменяйте пути к файлам
X_train = np.loadtxt(
    '/Users/yorko/Яндекс.Диск/ML/data/samsung_activities/samsung_train.txt')
y_train = np.loadtxt(
    '/Users/yorko/Яндекс.Диск/ML/data/samsung_activities/samsung_train_labels.txt').astype(int)
X_test = np.loadtxt(
    '/Users/yorko/Яндекс.Диск/ML/data/samsung_activities/samsung_test.txt')
y_test = np.loadtxt(
    '/Users/yorko/Яндекс.Диск/ML/data/samsung_activities/samsung_test_labels.txt').astype(int)

In [ ]:
X_train.shape, X_test.shape

**Выделим столько главных компонент, чтоб осталось 90% дисперсии исходных данных.**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

pca = PCA(0.9)
X_train_pca = pca.fit_transform(X_train_scaled)
print(X_train_pca.shape)

X_test_pca = pca.transform(scaler.transform(X_test))
print(X_test_pca.shape)

**Зафиксируем объект StratifiedKFold, с его помощью будет проводить кросс-валидацию.**

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)

**Используйте sklearn-обертку Xgboost для оценки доли правильных ответов на кросс-валидации с данными (X_train_pca, y_train). Укажите параметры XGBClassifier n_estimators = 10 (по умолчанию – 100) и seed=7 (это то же по сути, что random_state). Выведите среднюю долю правильных ответов на кросс-валидации, используйте ранее созданный объект StratifiedKFold.**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
xgb_model1 = XGBClassifier(n_estimators=10, seed=7)

In [ ]:
''' ВАШ КОД ЗДЕСЬ'''

**Постройте кривые валидации по глубине деревьев. Все прочие параметры кросс-валидации возьмите как на предыдущем шаге. Достаточно сохранять только среднюю долю правильных ответов на валидационной части (на обучающей – не обязательно).**

In [ ]:
%%time
tree_depths = [6, 9, 12, 15]
xgb_scores = []
for depth in tree_depths:
    print(depth, end=',')
    ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
plot(tree_depths, xgb_scores, color='green', label='Xgb');
xlabel('Max depth')
ylabel('3-CV accuracy')
legend();

**Используйте RandomizedGridSearch, чтобы настроить параметры бустинга. Задайте 10 итераций (будут перебираться случайные 10 комбинаций параметров), а кросс-валидацию – как ранее. Выведите лучшие параметры и accuracy (поля best\_params\_ и best\_score\_ у обученного RandomizedGridSearch). Сделайте то же самое для случайного леса.**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
xgb_params_dist_grid = {
    'max_depth': np.arange(10,14),
    'gamma': np.linspace(0, 0.5, 5),
    'subsample': [0.5, 0.75, 1.],
    'colsample_bytree': [0.5, 0.75, 1.]
}
xgb_params_fixed = {
    'n_estimators': 10,
    'silent': 1,
    'seed': 7
}

In [ ]:
xgb_grid = RandomizedSearchCV(
    ''' ВАШ КОД ЗДЕСЬ'''
)

In [ ]:
%%time
xgb_grid.fit(X_train_pca, y_train);

In [ ]:
xgb_grid.best_params_

In [ ]:
xgb_grid.best_score_

**Теперь для сравнения случайный лес.**

In [ ]:
rf_params_dist_grid = {
    'max_depth': np.arange(10,14),
    'max_features': np.linspace(0.2, 0.5, 10)
}
rf_params_fixed = {
    'n_estimators': 10,
    'random_state': 7
}

In [ ]:
rf_grid = RandomizedSearchCV(
    ''' ВАШ КОД ЗДЕСЬ'''
)

In [ ]:
%%time
rf_grid.fit(X_train_pca, y_train)

In [ ]:
rf_grid.best_params_, rf_grid.best_score_

**Проверьте доли правильных ответов настроенного и изначального бустинга, а также случайного леса на оставленной выборке.**

In [ ]:
# first xgb
xgb_model1.fit(X_train_pca, y_train)

In [ ]:
accuracy_score  ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
# tuned xgb
accuracy_score  ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
# tuned forest
accuracy_score  ''' ВАШ КОД ЗДЕСЬ'''

**Наконец, подберите число итераций бустинга. Постройте кривые валидации Xgboost с помощью функции cv из xgboost.**

In [ ]:
from xgboost import cv, DMatrix

In [ ]:
# каприз Xgboost: надо чтоб метки были от 0 до n_classes-1
y_train -= 1
y_test -= 1

In [ ]:
dtrain = DMatrix(X_train_pca, y_train)

In [ ]:
best_xgb_params = {'colsample_bytree': 0.75, 'gamma': 0.25, 
                   'max_depth': 11, 'subsample': 1.0,
                    'silent': 1, 
                   'eval_metric': 'merror',
                   'num_class': 6,
                    'seed': 7}

In [ ]:
# 20 iterations
cv_result = cv ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
cv_result.head()

In [ ]:
def plot_with_err(index, means, stds=None, **kwargs):
    lines = plt.plot(index, means, '-', **kwargs)
    plt.fill_between(index, means - stds, means + stds, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

In [ ]:
plot_with_err(cv_result.index, cv_result['train-merror-mean'], 
              cv_result['train-merror-std']);
plot_with_err(cv_result.index, cv_result['test-merror-mean'], 
              cv_result['test-merror-std']);

In [ ]:
%%time
# 100 iterations
cv_result2 = cv ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
plot_with_err(cv_result2.index, cv_result2['train-merror-mean'], 
              cv_result2['train-merror-std']);
plot_with_err(cv_result2.index, cv_result2['test-merror-mean'], 
              cv_result2['test-merror-std']);

In [ ]:
best_num_rounds = np.argmin(cv_result2['test-merror-mean'])

**Проверьте долю правильных ответов последней версии бустинга на оставленной выборке. Используйте функцию train из xgboost. X_test также надо будет переделать в DMatrix, только ответы y_test туда уже не передавайте.**

In [ ]:
from xgboost import train

In [ ]:
final_xgb = train ''' ВАШ КОД ЗДЕСЬ'''

In [ ]:
accuracy_score ''' ВАШ КОД ЗДЕСЬ'''